In [ ]:
import time
import numpy as np
import pandas as pd
 
from bs4 import BeautifulSoup
import urllib.request as req
 
import requests
import os
import datetime
import re

In [ ]:
# ディビジョンを選択してください:{v1_m, v2_m, v3_m, v1_w, v2_w}
division = 'v1_m'
# '年度を選択してください:{2019-20, 2018-19, 2017-18...}'
season = '2020-21'
# シーズンを選択してください:{regular, 2019-20(game1, game2, semi_final, final), 2014-15~2018-19(final6, final3, final),
# ~2013-14(semi_final, final)}
s_round = 'regular'
season += '_' + s_round
season

In [ ]:
# 2020-21の更新
os.chdir('/Users/sasno/Desktop/MyPandas/vleague')

headers = {"User-Agent": "Mozilla/5.0"}
 
# seasons = get_seasons(division)
 
# division = "v1_m"
# s_id = "283" # 2019-20
# s_id = "301"
division = "v1_w"
s_id = "303"
# division = "v2_m"
# s_id = "299"
# division = "v2_w"
# s_id = "302"
# division = "v3_m"
# s_id = "300"
# division = "v3_w"
# s_id = "268" 
s_round = "2019-20_regular"
 
if not os.path.isdir(division):
            os.makedirs(division)
 
os.chdir(division)
print(os.getcwd())
 
 
sets = ['1','2','3','4','5']
game_all = None
 
# ページ数を取得
url = 'https://www.vleague.jp/round/list/{}'.format(s_id)
print(url)
request = req.Request(url, headers=headers)
response = req.urlopen(request)
parse_html = BeautifulSoup(response, 'html.parser')
hrefs = parse_html.find_all('a', href=re.compile("pg"))
 
# for i in hrefs:
#     print(i.text)
if len(hrefs) > 0:
    pages = int(hrefs[-2].text)
    print(pages)
 
    all_list = []
    for pg in range(1,pages+1):
        url = 'https://www.vleague.jp/round/list/{0}?pg={1}'.format(s_id,pg)
        print(url)
        request = req.Request(url, headers=headers)
        response = req.urlopen(request)
        parse_html = BeautifulSoup(response, 'html.parser')
        tables = parse_html.find_all('table')
 
        url_list = []
        trs = tables[0].find_all('a', href=re.compile("/form/b"))
        for tr in trs:
            game_url = 'https://www.vleague.jp/'+ tr.attrs['href']
            url_list.append(game_url)
        print(url_list)
 
        for url in url_list:
            # print(url)
            html = requests.get(url, headers=headers)
            check = html.text
            if '試合データがありません' not in check:
                try:
                    data = pd.read_html(check, header=[2])
 
                    for j in range(2):
                        stats = pd.DataFrame(data[j+4])
                        stats = stats.rename(columns=str)
                        col = stats.columns.values
                        data_item = []
                        for i in col:
                            l = i
                            data_item.append(l)
                        for i in range(1,len(data_item)-2):
                            data_item[-i] = data_item[-i-3]
                            # print(data_item)
                        for i in range(3):
                        #     print(data_item[i])
                            if data_item[i] == '出場数':
                                data_item[i] = '背番号'
                            elif data_item[i] == '1':
                                data_item[i] = 'リベロ'
                            else:
                                data_item[i] = '名前'
                        #     print(i, data_item[i])
 
                        new_stats = stats.rename(columns={col[i]:data_item[i] for i in range(len(data_item))})
                        new_stats = new_stats.rename(columns={
                            '打数':'アタック打数', '得点':'アタック得点', '失点':'アタック失点', '決定率':'アタック決定率',
                            'セ平ット均':'アタックセット平均','打数.1':'バックアタック打数', '得点.1':'バックアタック得点',
                            '失点.1':'バックアタック失点', '決定率.1':'バックアタック決定率', '得点.2':'ブロック得点',
                            'セ平ット均.1':'ブロックセット平均', '打数.2':'サーブ打数', '得点.3':'サーブ得点', '失点.2':'サーブ失点',
                            '効果':'サーブ効果', '効果率':'サーブ効果率', '成功・優':'サーブレシーブ成功・優', 
                            '成功・良':'サーブレシーブ成功・良', '成功率':'サーブレシーブ成功率'
                            })
                        
                        request = req.Request(url, headers=headers)
                        response = req.urlopen(request)
                        parse_html = BeautifulSoup(response, 'html.parser')
                        table_ha = parse_html.find_all('table')[0]
                        td_ha = table_ha.find_all('td', class_='team')
                        # print(td_ha)
                        team = td_ha[j].text
                        # print('team={}'.format(team))
                        new_stats['アタック決定率'] = ((new_stats['アタック得点'] / new_stats['アタック打数']) * 100).round(1)
                        new_stats.insert(13, 'アタック効果率',
                            (((new_stats['アタック得点'] - new_stats['アタック失点']) / new_stats['アタック打数']) * 100).round(1))
                        new_stats['総得点'] = new_stats['アタック得点'] + new_stats['ブロック得点'] + new_stats['サーブ得点']
                        span = parse_html.find_all('span')
                        date = span[1].text.replace('/','-')
                        new_stats.insert(0, '試合日', date)
                        new_stats.insert(1, 'チーム', team)
                        new_stats['名前'] = new_stats['名前'].str.replace('\u3000','')
                        for by_set in sets:
                            new_stats[by_set] = new_stats[by_set].astype(str)
                        # 必要に応じてteamディレクトリを作成
                        team_dir = '{0}/{1}'.format(team,s_round)
                        if not os.path.isdir(team_dir):
                            os.makedirs(team_dir)
                        # new_stats.to_csv('{0}/{1}/{2}.csv'.format(team,s_round,date), index=False, encoding='cp932')
                        # print(new_stats)
                        all_list.append(new_stats)
                except:
                    pass
    game_all = pd.concat(all_list, ignore_index=True)
else:
    all_list = []
    url = 'https://www.vleague.jp/round/list/{}'.format(s_id)
    print(url)
    request = req.Request(url, headers=headers)
    response = req.urlopen(request)
    parse_html = BeautifulSoup(response, 'html.parser')
    tables = parse_html.find_all('table')
 
    url_list = []
    trs = tables[0].find_all('a', href=re.compile("/form/b"))
    for tr in trs:
        game_url = 'https://www.vleague.jp/'+ tr.attrs['href']
        url_list.append(game_url)
    print(url_list)
 
    for url in url_list:
        # print(url)
        html = requests.get(url, headers=headers)
        check = html.text
        if '試合データがありません' not in check:
            # try:
            data = pd.read_html(check, header=[2])
 
            for j in range(2):
                stats = pd.DataFrame(data[j+4])
                stats = stats.rename(columns=str)
                col = stats.columns.values
                data_item = []
                for i in col:
                    l = i
                    data_item.append(l)
                for i in range(1,len(data_item)-2):
                    data_item[-i] = data_item[-i-3]
                    # print(data_item)
                for i in range(3):
                #     print(data_item[i])
                    if data_item[i] == '出場数':
                        data_item[i] = '背番号'
                    elif data_item[i] == '1':
                        data_item[i] = 'リベロ'
                    else:
                        data_item[i] = '名前'
                #     print(i, data_item[i])
 
                new_stats = stats.rename(columns={col[i]:data_item[i] for i in range(len(data_item))})
                new_stats = new_stats.rename(columns={
                    '打数':'アタック打数', '得点':'アタック得点', '失点':'アタック失点', '決定率':'アタック決定率',
                    'セ平ット均':'アタックセット平均','打数.1':'バックアタック打数', '得点.1':'バックアタック得点',
                    '失点.1':'バックアタック失点', '決定率.1':'バックアタック決定率', '得点.2':'ブロック得点',
                    'セ平ット均.1':'ブロックセット平均', '打数.2':'サーブ打数', '得点.3':'サーブ得点', '失点.2':'サーブ失点',
                    '効果':'サーブ効果', '効果率':'サーブ効果率', '成功・優':'サーブレシーブ成功・優', 
                    '成功・良':'サーブレシーブ成功・良', '成功率':'サーブレシーブ成功率'
                    })
                
                request = req.Request(url, headers=headers)
                response = req.urlopen(request)
                parse_html = BeautifulSoup(response, 'html.parser')
                table_ha = parse_html.find_all('table')[0]
                td_ha = table_ha.find_all('td', class_='team')
                # print(td_ha)
                team = td_ha[j].text
                # print('team={}'.format(team))
                new_stats['アタック決定率'] = ((new_stats['アタック得点'] / new_stats['アタック打数']) * 100).round(1)
                new_stats.insert(13, 'アタック効果率', ((new_stats['アタック得点'] - new_stats['アタック失点']) / new_stats['アタック打数'] * 100).round(1))
                new_stats['総得点'] = new_stats['アタック得点'] + new_stats['ブロック得点'] + new_stats['サーブ得点']
                span = parse_html.find_all('span')
                date = span[1].text.replace('/','-')
                new_stats.insert(0, '試合日', date)
                new_stats.insert(1, 'チーム', team)
                new_stats['名前'] = new_stats['名前'].str.replace('\u3000','')
                for by_set in sets:
                    new_stats[by_set] = new_stats[by_set].astype(str)
                必要に応じてteamディレクトリを作成
                team_dir = '{0}/{1}'.format(team,s_round)
                if not os.path.isdir(team_dir):
                    os.makedirs(team_dir)
                new_stats.to_csv('{0}/{1}/{2}.csv'.format(team,s_round,date), index=False, encoding='cp932')
                all_list.append(new_stats)
            # except:
            #     pass
    try:
        game_all = pd.concat(all_list, ignore_index=True)
        print(game_all)
 
    except:
        pass
all_dir = 'all/{}'.format(s_round)
if not os.path.isdir(all_dir):
    os.makedirs(all_dir)
 
print(os.getcwd())
try:
    daily_all = game_all[game_all['名前'] != 'チーム合計']
    print(daily_all)
 
    data = daily_all
    data.insert(1, '月', data['試合日'].str[5:7])
    data.set_index('試合日')
    monthly_all = data.groupby(['名前','月','チーム','背番号']).sum().reset_index()
    monthly_all['アタック決定率'] = ((monthly_all['アタック得点'] / monthly_all['アタック打数']) * 100).round(1)
    monthly_all['アタック効果率'] = (((monthly_all['アタック得点']-monthly_all['アタック失点']) / monthly_all['アタック打数']) * 100).round(1)
    monthly_all['バックアタック決定率'] = ((monthly_all['バックアタック得点'] / monthly_all['バックアタック打数']) * 100).round(1)
    monthly_all['アタックセット平均'] = (monthly_all['アタック得点'] / monthly_all['出場数']).round(1)
    monthly_all['ブロックセット平均'] = (monthly_all['ブロック得点'] / monthly_all['出場数']).round(1)
    monthly_all['サーブ効果率'] = ((monthly_all['サーブ得点'] * 100 + monthly_all['サーブ効果'] * 25 - monthly_all['サーブ失点'] * 25) / monthly_all['サーブ打数']).round(1)
    monthly_all['サーブレシーブ成功率'] = ((monthly_all['サーブレシーブ成功・優'] * 100 + monthly_all['サーブレシーブ成功・良'] * 50) / monthly_all['受数']).round(1)
    print(monthly_all)
 
    yearly_all = data.groupby(['名前','チーム','背番号']).sum().reset_index()
    yearly_all['アタック決定率'] = ((yearly_all['アタック得点'] / yearly_all['アタック打数']) * 100).round(1)
    yearly_all['アタック効果率'] = (((yearly_all['アタック得点']-yearly_all['アタック失点']) / yearly_all['アタック打数']) * 100).round(1)
    yearly_all['バックアタック決定率'] = ((yearly_all['バックアタック得点'] / yearly_all['バックアタック打数']) * 100).round(1)
    yearly_all['アタックセット平均'] = (yearly_all['アタック得点'] / yearly_all['出場数']).round(1)
    yearly_all['ブロックセット平均'] = (yearly_all['ブロック得点'] / yearly_all['出場数']).round(1)
    yearly_all['サーブ効果率'] = ((yearly_all['サーブ得点'] * 100 + yearly_all['サーブ効果'] * 25 - yearly_all['サーブ失点'] * 25) / yearly_all['サーブ打数']).round(1)
    yearly_all['サーブレシーブ成功率'] = ((yearly_all['サーブレシーブ成功・優'] * 100 + yearly_all['サーブレシーブ成功・良'] * 50) / yearly_all['受数']).round(1)
    print(yearly_all)
 
    game_all.to_csv('all/{}/game_all.csv'.format(s_round), index=False, encoding='cp932')
    daily_all.to_csv('all/{}/daily_all.csv'.format(s_round), index=False, encoding='cp932')
    monthly_all.to_csv('all/{}/monthly_all.csv'.format(s_round), index=False, encoding='cp932')
    yearly_all.to_csv('all/{}/yearly_all.csv'.format(s_round), index=False, encoding='cp932')
    print('csv作成成功')
except:
    pass

/content/drive/My Drive/vleague/v1_w
https://www.vleague.jp/round/list/303
7
https://www.vleague.jp/round/list/303?pg=1
['https://www.vleague.jp//form/b/25907', 'https://www.vleague.jp//form/b/25905', 'https://www.vleague.jp//form/b/25901', 'https://www.vleague.jp//form/b/25934', 'https://www.vleague.jp//form/b/25902', 'https://www.vleague.jp//form/b/25906', 'https://www.vleague.jp//form/b/25903', 'https://www.vleague.jp//form/b/25908', 'https://www.vleague.jp//form/b/25904', 'https://www.vleague.jp//form/b/25912', 'https://www.vleague.jp//form/b/25915', 'https://www.vleague.jp//form/b/25913', 'https://www.vleague.jp//form/b/25909', 'https://www.vleague.jp//form/b/25936', 'https://www.vleague.jp//form/b/25910', 'https://www.vleague.jp//form/b/25914', 'https://www.vleague.jp//form/b/25916', 'https://www.vleague.jp//form/b/25911', 'https://www.vleague.jp//form/b/25923', 'https://www.vleague.jp//form/b/25921']
https://www.vleague.jp/round/list/303?pg=2
['https://www.vleague.jp//form/b/259

In [ ]:
import tweepy
import os
import glob
import time
import numpy as np
import pandas as pd
import re
 
# key
consumer_key = 'D4OAzhqT7NNeeCa9R7h0Hxzx3'
consumer_secret = 'Py8P1k5jxdPEvNOb64WUXh8b10sfDL5qr60krVLNOOLEXauZti'
access_token = '1309887734555668481-NNvdk0MCxK4WwKZCKZrOqflU9bwPfG'
access_secret = 'CmkMLM4eiOq4MLP3TE6FfUobThOElR5OAfuVSv3GRnXrJ'
 
# Twitterオブジェクト
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit = True)

In [ ]:
daily_all = pd.read_csv('all/{}/daily_all.csv'.format(s_round),encoding='cp932')
# daily_all
day = '2020-10-18'
today_data = daily_all[daily_all['試合日']==day]
# today_data.head()
stats_list = ['total', 'attack', 'b_attack', 'block', 'surve', 'cut']
# stats = np.random.choice(stats_list)
for stats in stats_list:
    print(stats)
    if stats == 'total':
        reg_judge = today_data['出場数']>=1
        column = '総得点'
        pattern = ['出場数1セット', 0]
    elif stats == 'attack':
        reg_judge = today_data['アタック打数']>=10
        column = 'アタック決定率'
        pattern = ['打数10本', 1]
    elif stats == 'b_attack':
        reg_judge = today_data['バックアタック打数']>=5
        column = 'バックアタック決定率'
        pattern = ['打数5本', 1]
    elif stats == 'block':
        reg_judge = today_data['出場数']>=1
        column = 'ブロックセット平均'
        pattern = ['出場数1セット', 0]
    elif stats == 'surve':
        reg_judge = today_data['サーブ打数']>=5
        column = 'サーブ効果率'
        pattern = ['サーブ打数5本', 1]
    else:
        reg_judge = today_data['受数']>=5
        column = 'サーブレシーブ成功率'
        pattern = ['受数5本', 1]
    print(column)

    for i in range(3):
        stats_df = today_data[reg_judge][['名前', 'チーム', '背番号', '出場数', column]]
        stats_df = stats_df.sort_values(column, ascending=False).head()
        print(stats_df)
        if len(stats_df) >= 3:
            break
    # try:
    # stats_df = stats_df.to_dict()
    print(stats_df)
    p = re.compile('[\u0000-\u007F]+')
    messages = '本日({})の成績ランキング\n'.format(day)
    messages += 'シーズン: {}\n'.format(season)
    messages += '{0}ランキング({1}以上)\n'.format(column, pattern[0])
    messages += 'ディビジョン: {}\n'.format(division)
    print(len(stats_df))
    for i in range(len(stats_df)):
        if pattern[1]:
            message = '{0}位 {1} ({2}, {3}%)\n'.format(i+1, stats_df['名前'].iloc[i], stats_df['チーム'].iloc[i], stats_df[column].iloc[i])
        else:
            message = '{0}位 {1} ({2}, {3})\n'.format(i+1, stats_df['名前'].iloc[i], stats_df['チーム'].iloc[i], stats_df[column].iloc[i])
        m_len = messages + message
        m_len = messages + message
        e_len = len(''.join(p.findall(m_len)))
        tweet_len = len(m_len) - (e_len/2)
        # print(tweet_len)
        if tweet_len <= 140:
            messages += message
        else:
            break
    print(messages)
    api.update_status(messages)
    # except:
    #     pass

total
総得点
             名前              チーム  背番号  出場数  総得点
150  オクム大庭冬美ハウィ          日立リヴァーレ   17    5   40
189   ドルーズアンドレア         JTマーヴェラス   10    4   27
246        山内美咲       NECレッドロケッツ   12    4   25
267        舛田紗淑  KUROBEアクアフェアリーズ    9    4   25
230        金田修佳          岡山シーガルズ   18    4   22
             名前              チーム  背番号  出場数  総得点
150  オクム大庭冬美ハウィ          日立リヴァーレ   17    5   40
189   ドルーズアンドレア         JTマーヴェラス   10    4   27
246        山内美咲       NECレッドロケッツ   12    4   25
267        舛田紗淑  KUROBEアクアフェアリーズ    9    4   25
230        金田修佳          岡山シーガルズ   18    4   22
5
本日(2020-10-18)の成績ランキング
シーズン: 2020-21_regular
総得点ランキング(出場数1セット以上)
ディビジョン: v1_w
1位 オクム大庭冬美ハウィ (日立リヴァーレ, 40)
2位 ドルーズアンドレア (JTマーヴェラス, 27)
3位 山内美咲 (NECレッドロケッツ, 25)

attack
アタック決定率
                 名前          チーム  背番号  出場数  アタック決定率
169            岩坂名奈     久光スプリングス    4    4     60.0
215             黒後愛       東レアローズ    9    3     57.1
250  サンティアゴアライジャダフニ   埼玉上尾メディックス    3    4     56.0
159           荒木絵里香  トヨタ車体クイン

In [ ]:
daily_all = pd.read_csv('all/2019-20_regular/daily_all.csv',encoding='cp932')
daily_all.sort_values('総得点', ascending=False).head()

,試合日,月,チーム,背番号,リベロ,名前,出場数,1,2,3,4,5,アタック打数,アタック得点,アタック失点,アタック決定率,アタック効果率,アタックセット平均,バックアタック打数,バックアタック得点,バックアタック失点,バックアタック決定率,ブロック得点,ブロックセット平均,サーブ打数,サーブ得点,サーブ失点,サーブ効果,サーブ効果率,受数,サーブレシーブ成功・優,サーブレシーブ成功・良,サーブレシーブ成功率,総得点
3630,2020-02-15,2,堺ブレイザーズ,9,NaN,モーリストーレス,5,■,■,■,■,■,67,39,4,58.2,52.2,7.80,24,14,2,58.3,2,0.40,18,1,4,6,8.3,0,0,0,-,42
1813,2019-12-14,12,サントリーサンバーズ,13,NaN,ムセルスキードミトリー,5,■,■,■,■,■,56,31,7,55.4,42.9,6.20,23,16,2,69.6,6,1.20,20,2,8,4,5.0,3,0,0,0.0,39
1072,2019-11-23,11,サントリーサンバーズ,13,NaN,ムセルスキードミトリー,4,■,■,■,■,NaN,45,32,2,71.1,66.7,8.00,16,12,0,75.0,3,0.75,13,3,5,2,17.3,0,0,0,-,38
1578,2019-12-07,12,FC東京,20,NaN,プレモビッチピーター,4,■,■,■,■,NaN,66,32,2,48.5,45.5,8.00,36,18,2,50.0,4,1.00,22,1,4,6,6.8,0,0,0,-,37
1368,2019-11-30,11,ジェイテクトSTINGS,14,NaN,西田有志,5,■,■,■,■,■,51,30,3,58.8,52.9,6.00,25,13,1,52.0,3,0.60,22,4,6,11,23.9,1,0,0,0.0,37
